# YamboWorkflow: a full DFT+MBPT flow

This is the core workflow of the plugin. It runs a DFT+GW/BSE study, involving automatic error handling and skipping the already done steps - if a parent calculation is provided.

The idea is to create a builder instance, which collects all the inputs needed for the simulation, then submit it by means of the aiida "submit" function.

In [1]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

## 1 - Creation of YamboWorkflow instance using protocols

In this tutorial the creation of the builder, for a given workflow, is provided by means of the get_builder_from_protocol function.
This is, for now, just a way to automatically populated all the needed inputs, allowing the user to partially skip the long part of input definition as done in examples like the one contained in ``aiida_yambo/examples_hBN/workflows/yambo_workflow.py``.


It is not meant to be an already tested set of default values as instead is done for the aiida-quantumespresso plugin. 

In [2]:
from aiida.plugins import WorkflowFactory

In [3]:
YamboWorkflow = WorkflowFactory('yambo.yambo.yambowf')

## minimal inputs needed for protocols

In [4]:
options = {
    'pwcode_id': 'pw-7.1@hydralogin', 
    'pseudo_family':"PseudoDojo/0.4/PBE/FR/standard/upf",
    'yamboprecode_id':'p2y-5.1@hydralogin',
    'yambocode_id':'yambo-5.1@hydralogin',
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; take your previously nscf id (pk) to skip the DFT part.
    'structure_id':161,
}

In [5]:
#YamboWorkflow.get_builder_from_protocol??

In [6]:
from aiida_quantumespresso.common.types import ElectronicType

In [7]:
builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides={},
            pseudo_family= options['pseudo_family'],
            #parent_folder=orm.load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='gw', #or 'bse'; default is 'gw'
)


Summary of the main inputs:
BndsRnXp = 200
GbndRnge = 200
NGsBlkXp = 6 Ry
FFTGvecs = 18 Ry


kpoint mesh for nscf: [6, 6, 2]


With respect to the previous examples (2_YamboRestart), we can see that we have a different FFTGvecs value and a kpoint mesh: the first is due to the fact that now FFTGvecs is estimated from aiida-quantumespresso protocol, not from the previous DFT run. The second is needed in order to have also the DFT part performed, if needed (no previous parent calculations).

In [8]:
#You can also try different protocols:
    
YamboWorkflow.get_available_protocols()

{'fast': {'description': 'Under converged for most materials, but fast'},
 'moderate': {'description': 'Meta converged for most materials, higher computational cost than fast'},
 'precise': {'description': 'Converged for most materials, higher computational cost than moderate'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [9]:
builder.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

In [10]:
builder.yres.yambo.parameters.get_dict()

{'arguments': ['dipoles', 'ppa', 'HF_and_locXC', 'gw0'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'GTermKind': 'BG',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'NGsBlkXp': [6, 'Ry'],
  'FFTGvecs': [18, 'Ry'],
  'BndsRnXp': [[1, 200], ''],
  'GbndRnge': [[1, 200], ''],
  'QPkrange': [[[1, 1, 32, 32]], '']}}

## 2 - Inputs completion

We have to include also the resources:

In [11]:
builder.scf.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.nscf.pw.metadata.options = builder.scf.pw.metadata.options
builder.yres.yambo.metadata.options = builder.scf.pw.metadata.options

## 3 - Overrides + Spin Orbit Coupling

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [12]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/FR/standard/upf", 
        'pw':{
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{
                    "noncolin": True,
                    "lspinorb": True,
                },
                'ELECTRONS':{},
            },
            
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/FR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{
                    "noncolin": True,
                    "lspinorb": True,
                },
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "NGsBlkXp": [4, "Ry"],
                    "FFTGvecs": [20, "Ry"],
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

overrides = {
    'yres': overrides_yambo,
    'nscf': overrides_nscf,
    'scf': overrides_scf
    
}


In [13]:
builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='gw', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXp = 200
GbndRnge = 200
NGsBlkXp = 4 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


In [14]:
builder.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'noncolin': True,
  'lspinorb': True,
  'ecutwfc': 84.0,
  'ecutrho': 336.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'diagonalization': 'cg',
  'conv_thr': 1.6e-09}}

In [15]:
builder.yres.yambo.metadata.options

{'stash': {}, 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 16, 'num_cores_per_mpiproc': 1}, 'max_wallclock_seconds': 3600, 'withmpi': True, 'prepend_text': 'export OMP_NUM_THREADS=1', 'queue_name': 's3par'}

In [16]:
builder.nscf.pw.parameters.get_dict()['ELECTRONS']['diagonalization']

'cg'

In [17]:
family = orm.load_group("PseudoDojo/0.4/PBE/FR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 
builder.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

## 4 - explicit request calculation for the band gap

Here an explanation of how to ask other gaps.

In [18]:
builder.additional_parsing = orm.List(list=['gap_GG','gap_'])

## 5 RUN

In [19]:
from aiida.engine import submit

In [20]:
run = None

In [21]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 8c450461-422a-481f-bb4e-7f6f2a5fcb62 (pk: 2433) (aiida.workflows:yambo.yambo.yambowf)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance. All the outputs of YamboRestart and YamboCalculation are inherited

In [22]:
run.is_finished_ok

True

In [23]:
run.outputs.output_ywfl_parameters.get_dict()

{'gap_': 5.3789906471538,
 'homo': -0.20861223974703,
 'lumo': 5.1703784074068,
 'gap_GG': 7.3148622832797,
 'homo_G': -1.5859060006194,
 'lumo_G': 5.7289562826604,
 'gap_dft': 4.2862447410166,
 'homo_dft': -8.5151106119156e-06,
 'lumo_dft': 4.286236225906,
 'gap_GG_dft': 6.7760488920897,
 'homo_G_dft': -1.2751118633211,
 'lumo_G_dft': 5.5009370287687}

For example, we have the array_ndb ArrayData, which essentially contains the information on the run.output.QP_DB (which can also be accessed directly):

In [24]:
run.outputs.nscf_mapping.get_dict()

{'soc': True,
 'gap_': [[14, 14, 15, 15], [7, 7, 17, 17]],
 'gap_GG': [[1, 1, 15, 15], [1, 1, 17, 17]],
 'homo_k': 14,
 'lumo_k': 7,
 'valence': 15,
 'gap_type': 'indirect',
 'conduction': 17,
 'nscf_gap_eV': 4.286,
 'dft_predicted': 'semiconductor/insulator',
 'spin-resolved': False,
 'number_of_kpoints': 14}

In [33]:
# some comments on the results.